# Micro-Grad like Engine

In [73]:
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
import random

In [105]:
class Value:

    def __init__(self, data, label = '', parents = (), _op = ''):
        # Gradient
        self.grad = 0.0
        # Backward fn:
        self._backward = lambda: None
        self.data = data
        self._label = label
        self._parents = set(parents)
        self._op = _op
    
    def __repr__(self):
        if self._label:
            return f"{self._label}: Value(data={self.data})"
        else:
            return f"Value(data={self.data})"


    def set_label(self, label = ''):
        self._label = label
    
    def get_label(self):
        return self._label

    
    # overwrite math operation:
    def __add__ (self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, parents= (self, other), _op='+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out
    
    def __sub__(self, other): # self - other
        return self + (-other)
    
    def __mul__ (self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, parents=(self, other), _op='*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward

        return out
    
    def __neg__(self): # -self
        return self * Value(-1)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def backward(self):
    
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for parent in v._parents:
                    build_topo(parent)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()



In [87]:
a = Value(5, label='a')
b = Value(2, label='b')
e = Value(3, 'e')

In [88]:
l1 = b * e
c = a + l1
c.set_label('c')

In [89]:
c

c: Value(data=11)

In [90]:
c.backward()

In [91]:
a.grad

1.0

In [92]:
b.grad

3.0

In [93]:
e.grad

2.0

In [94]:
class Neuron:

    def __init__(self, ninput):
        self.w = [Value(random.uniform(-1,1)) for _ in range(ninput)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self, x):
        # out = act(wx+b)
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
  
    def parameters(self):
        return self.w + [self.b]


In [95]:
class Layer:
    def __init__(self, ninput, noutput):
        self.neurons = [Neuron(ninput) for _ in range(noutput)]
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
  
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

In [96]:
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
  
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [97]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
out = n(x)

In [98]:
type(out)

__main__.Value

In [106]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [Value(1.0), Value(-1.0), Value(-1.0), Value(1.0)] # desired targets

In [107]:
for k in range(20):
  
  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
  # backward pass
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()
  
  # update
  for p in n.parameters():
    p.data += -0.1 * p.grad
  
  print(k, loss.data)

TypeError: unsupported operand type(s) for +: 'int' and 'Value'